In [1]:
import sys
sys.path.append('../')
"""
2024-11-26
생성 목적.
1. OMS나 ClangCursor를 통한 태스트 데이터 셋 생성
2. LongCoder 재 확인.

"""

'\n2024-11-26\n생성 목적.\n1. OMS나 ClangCursor를 통한 태스트 데이터 셋 생성\n2. LongCoder 재 확인.\n\n'

In [2]:
import time
import os

from oms.Mapper import parsing
#일단 태스트 용도로 간단하게 하나의 파일로. 또 태스트 데이터 생성을 위해 완전한(프로젝트) 파일로.
file_path = r'D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\UIDlgImplantLib.cpp'
st = time.time()
oms_info, clang_src_map = parsing(file_path)
ed = time.time()

print("project parse time :", (ed-st), "seconds")

print(len(clang_src_map), ' clang src map size')
print(oms_info)

#unit도 필요한듯
from clangParser.CUnit import CUnit
my_unit = CUnit.parse(file_path=file_path)




Mapping (100.00%) 101/101
Mapping done
Update Call (100.00%) 100/100
Update done
project parse time : 55.86138319969177 seconds
820  clang src map size
InfoSet(cls:78, fun:254, var:164)


In [3]:
#적당히 짧은 메서드들만 
from oms.dataset.info_base import InfoBase
from clangParser.Cursor import Cursor
short_method_list = []
all_method_list = []
gen_src_key_list = []

def get_gen_src_key(info: InfoBase):
    clang_list = clang_src_map[info.src_name]
    call_trace=Cursor(clang_list[0]).get_line_call_trace()
    codes = ''
    for code, src_set in call_trace:
        for src in src_set:
            codes += src + ' '
        codes += '\n'
    # print([node.location.file for node in clang_list])
    return codes


cls_info:InfoBase = oms_info.get_class_info('CUIDlgImplantLib')
for fun_info in cls_info.relationInfo.hasInfoMap.functionInfos.values():
    all_method_list.append(fun_info)
    if 15  < len(fun_info.code.splitlines()):
        continue
    else:
        short_method_list.append(fun_info)

#설정된 메서드 부분 출력
print(cls_info.relationInfo.hasInfoMap)
print('->' , len(short_method_list))
for idx, fun in enumerate(short_method_list):
    if idx %15 == 0:
        codes = fun.code.splitlines()
        srcs = get_gen_src_key(fun).splitlines()
        for idx, src in enumerate(srcs):
            print(codes[idx], '\t', src)
            

InfoSet(cls:0, fun:99, var:41)
-> 46
65 - 65
455 - 461
bool CUIDlgImplantLib::OnMessageSpecificAction(UINT nAction, LPARAM lParam) 	 CUIDlgImplantLib ..LPARAM .UINT CUIDlgImplantLib.OnMessageSpecificAction(UINT, LPARAM).lParam CUIDlgImplantLib.OnMessageSpecificAction(UINT, LPARAM) CUIDlgImplantLib.OnMessageSpecificAction(UINT, LPARAM).nAction 
{ 	 
	// nothing to do, now. 	 
	UNREFERENCED_PARAMETER(nAction); 	 CUIDlgImplantLib.OnMessageSpecificAction(UINT, LPARAM).nAction 
	UNREFERENCED_PARAMETER(lParam); 	 CUIDlgImplantLib.OnMessageSpecificAction(UINT, LPARAM).lParam 
	return false; 	 
1880 - 1893
CString CUIDlgImplantLib::_getSystem() 	 CUIDlgImplantLib CUIDlgImplantLib._getSystem() CString 
{ 	 
	CString strSystem; 	 CString CUIDlgImplantLib._getSystem().strSystem 
	if (DCS_PRESSED == _btnTabFixture.GetState()) 	 E_DOCTRL_STATES.DCS_PRESSED CUIDlgImplantLib._btnTabFixture 
	{ 	 
		strSystem = _strFixtureSystem; 	 CUIDlgImplantLib._strFixtureSystem CUIDlgImplantLib._getSystem().strSy

In [4]:
# !wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

In [15]:
#일단 t5로 먼저 학습
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')
model.to('cuda')


T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [6]:
all_code_list = [fun_info.code for fun_info in short_method_list]
all_ans_list = [get_gen_src_key(fun_info) for fun_info in short_method_list]
# all_tokens_list = [tokenizer.tokenize(code_snippet) for code_snippet in all_code_list]
# all_tokens_ids_list = [torch.tensor(tokenizer.convert_tokens_to_ids(tokens)) for tokens in all_tokens_list]

65 - 65
71 - 79
84 - 95
100 - 103
108 - 108
108 - 118
169 - 175
177 - 182
276 - 281
283 - 295
399 - 404
409 - 417
422 - 428
433 - 439
444 - 450
455 - 461
466 - 472
477 - 483
492 - 501
505 - 511
513 - 522
525 - 532
536 - 544
549 - 559
798 - 806
808 - 818
1823 - 1837
1858 - 1861
1863 - 1866
1868 - 1878
1880 - 1893
2502 - 2516
2741 - 2751
2753 - 2756
2758 - 2761
2763 - 2766
2768 - 2771
2773 - 2776
2778 - 2781
2783 - 2786
2788 - 2791
2793 - 2796
3069 - 3076
3417 - 3426
3644 - 3647
3649 - 3652


In [7]:
batch_input = tokenizer(all_code_list, padding=True, truncation=True, return_tensors='pt')
batch_ans= tokenizer(all_ans_list, padding=True, truncation=True, return_tensors='pt')

print(batch_input["input_ids"].shape)  # 입력 토큰의 ID
print(batch_ans["input_ids"].shape)  # 출력 토큰의 ID

torch.Size([46, 258])
torch.Size([46, 512])


In [8]:
# #토큰화 된 데이터 재 출력
# for idx, tk in enumerate(batch_input['input_ids'][30]):
#     print(idx,":",tokenizer.decode(tk))

print(type(batch_input))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [9]:
# !pip install datasets

In [10]:
#dataset 생성
from datasets import Dataset

#먼저 dict으로
data_dict = {
    "input_ids": batch_input["input_ids"],
    "attention_mask": batch_input["attention_mask"],
    "labels": batch_ans["input_ids"]
}

# Dataset 생성
dataset = Dataset.from_dict(data_dict)
test_1_datas = dataset.train_test_split(test_size = 0.1, seed = 0)
print(test_1_datas)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 41
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
})


In [30]:
# print(test_1_datas['test'][0])
print(tokenizer.decode(test_1_datas['test'][0]['labels']))

<s>CUIDlgImplantLib CUIDlgImplantLib.~CUIDlgImplantLib() 

CUIDlgImplantLib._pImgTitle 

CUIDlgImplantLib._pListManufacture 
CUIDlgImplantLib._pListSystem 
CUIDlgImplantLib._pListAbutment 

CUIDlgImplantLib._pListModel 

CUIDlgImplantLib._pDlgPreview 
</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [11]:
# pip show transformers
import transformers
print(transformers.__version__)


4.46.3


In [13]:
test_1_datas

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 41
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
})

In [16]:
from transformers import Trainer, Seq2SeqTrainingArguments

# Seq2SeqTrainingArguments 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",                   # 결과 저장 경로
    evaluation_strategy="epoch",             # 평가 빈도 (매 epoch마다)
    learning_rate=5e-5,                       # 학습률
    per_device_train_batch_size=8,            # 학습 배치 크기
    per_device_eval_batch_size=8,             # 평가 배치 크기
    num_train_epochs=5,                       # 학습 에포크 수
    weight_decay=0.01,                        # 가중치 감소 (정규화)
    save_total_limit=2,                       # 저장할 체크포인트 개수 제한
    save_strategy="epoch",                    # 체크포인트 저장 전략
    logging_dir="./logs",                     # 로깅 경로
    logging_steps=100,                        # 로깅 빈도
    predict_with_generate=True,               # 생성 기반 평가 활성화
    generation_max_length=128,                # 생성 최대 길이
    generation_num_beams=5                    # 빔 서치 빔 개수
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_1_datas["train"],
    eval_dataset=test_1_datas["test"]
)

# 학습 시작
trainer.train()

C:\dev\Anaconda\envs\T1\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.979421
2,No log,2.233285
3,No log,0.349738
4,No log,0.345459
5,No log,0.340202


TrainOutput(global_step=30, training_loss=0.8403551737467448, metrics={'train_runtime': 51.6491, 'train_samples_per_second': 3.969, 'train_steps_per_second': 0.581, 'total_flos': 13980913827840.0, 'train_loss': 0.8403551737467448, 'epoch': 5.0})

In [20]:
print(type(outputs.predictions))  # 데이터 타입
print(outputs.predictions)       # 데이터 내용


<class 'tuple'>
(array([[[ 1.55565901e+01,  3.06917591e+01,  4.65232849e+00, ...,
          2.41248035e+00,  2.31740689e+00,  7.27300787e+00],
        [ 1.23878920e+00,  9.43407917e+00,  9.51668262e+00, ...,
          1.06006565e+01,  1.08723860e+01,  2.37797527e+01],
        [-2.06121325e-01,  4.58405876e+00,  4.56489277e+00, ...,
          3.77803397e+00,  4.04435921e+00,  2.59056163e+00],
        ...,
        [ 2.65522118e+01,  6.12792492e+00,  3.57215452e+00, ...,
          1.68595576e+00,  2.91011715e+00,  1.52811849e+00],
        [ 2.65503445e+01,  6.12779093e+00,  3.57224011e+00, ...,
          1.68658459e+00,  2.91057515e+00,  1.52854395e+00],
        [ 2.65484486e+01,  6.12765503e+00,  3.57231736e+00, ...,
          1.68720424e+00,  2.91101456e+00,  1.52895439e+00]],

       [[ 1.55934525e+01,  3.04603577e+01,  4.58670855e+00, ...,
          2.32590532e+00,  2.23005128e+00,  7.16310072e+00],
        [ 1.12821817e+00,  9.24745846e+00,  9.35002136e+00, ...,
          1.06320906e

In [25]:
import torch

# 데이터셋에서 입력 텍스트 가져오기
test_input_ids = test_1_datas["test"]["input_ids"]
test_attention_mask = test_1_datas["test"]["attention_mask"]

# PyTorch 텐서로 변환
input_ids = torch.tensor(test_input_ids).to(model.device)
attention_mask = torch.tensor(test_attention_mask).to(model.device)

# 예측 토큰 ID 생성
generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=5)

# 디코딩하여 텍스트 변환
predictions = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
print(predictions)


['CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplant', 'CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._createImage() CUIDlgImplantLib._', 'BEGIN_MESSAGE_MAP(CUIDlgImplantLib, CUIDialog) BEGIN_MESSAGE_MAP(CUIDlgImplantLib, CUIDialog) END_MESSAGE_MAP(CUIDlgImplantLib, CUIDialog) BEGIN_MESSAGE_MAP(CUIDlgImplantLib, CUIDialog) END_MESSAGE_MAP(CUIDlgImplantLib, CUIDialog) END_MESSAGE_MAP(CUIDlgImplantLib', 'CUIDlgImplantLib.OnLButtonDown(UINT, CPoint).CUIDlgImplantLib.OnLButtonDown(UINT, CPoint).CUIDlgImplantLib.OnLButtonDown(UINT, CPoint)

In [40]:
#평가 시 생성된 결과 출력
outputs = trainer.predict(test_dataset=test_1_datas["test"])
# print(outputs.predictions)

print(type(outputs))
print('label ids')
print(type(outputs.label_ids), "\t", outputs.label_ids.shape)

print('\npredictions')
pr_types = [f'{type(d)} {d.shape}' for d in outputs.predictions]
print(type(outputs.predictions), "\t", pr_types)

# import numpy as np

# # 로짓에서 가장 높은 확률의 토큰 선택
# logits = outputs.predictions  # (batch_size, sequence_length, vocab_size)
# token_ids = np.argmax(logits, axis=-1)  # 각 위치에서 확률이 가장 높은 토큰 ID 선택

# # 디코딩
# predictions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in token_ids]
# print(predictions)



<class 'transformers.trainer_utils.PredictionOutput'>
label ids
<class 'numpy.ndarray'> 	 (5, 512)
predictions
<class 'tuple'> 	 ["<class 'numpy.ndarray'> (5, 512, 32100)", "<class 'numpy.ndarray'> (5, 258, 512)"]


In [51]:
tokenizer.get_vocab

<bound method RobertaTokenizer.get_vocab of RobertaTokenizer(name_or_path='Salesforce/codet5-small', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<extra_id_99>', '<extra_id_98>', '<extra_id_97>', '<extra_id_96>', '<extra_id_95>', '<extra_id_94>', '<extra_id_93>', '<extra_id_92>', '<extra_id_91>', '<extra_id_90>', '<extra_id_89>', '<extra_id_88>', '<extra_id_87>', '<extra_id_86>', '<extra_id_85>', '<extra_id_84>', '<extra_id_83>', '<extra_id_82>', '<extra_id_81>', '<extra_id_80>', '<extra_id_79>', '<extra_id_78>', '<extra_id_77>', '<extra_id_76>', '<extra_id_75>', '<extra_id_74>', '<extra_id_73>', '<extra_id_72>', '<extra_id_71>', '<extra_id_70>', '<extra_id_69>', '<extra_id_68>', '<extra_id_67>', '<extra_id_66>', '<extra_id_65>', '<extr

In [58]:
code ='''
BOOL CUIDlgImplantLib::OnInitDialog()
{
	CToolbarModuleBase::OnInitDialog();

	// TODO:  여기에 추가 초기화 작업을 추가합니다.
	
	_createImplantLibraryDlg();
	_createGroupDlg();

	CreateSlider();

	_pModuleDocAPImplantSimulation = reinterpret_cast<CModuleDocAPImplantSimulation*>(_pLocalDoc->GetWritableModuleDoc(APMod_ImplantSimul));
	_pModuleDocAPImplantSimulation->CheckImplantXml();

	InitializeToothBtnFlags();

	ShowUndoRedoWindow(TRUE);

	return TRUE;  // return TRUE unless you set the focus to a control
				  // 예외: OCX 속성 페이지는 FALSE를 반환해야 합니다.
}
'''

# 토크나이징
input_ids = tokenizer(code, return_tensors="pt", truncation=True, padding="max_length", max_length=128).input_ids

#디코딩
output_text = tokenizer.decode(generated_ids[0])

print(type(generated_ids[0]), generated_ids[0].shape)
print("Generated Output:", output_text)
output_text

<class 'torch.Tensor'> torch.Size([128])
Generated Output: <pad><s><extra_id_0>CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplant


'<pad><s><extra_id_0>CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplantLib.~CUIDlgImplant'

In [66]:

tokens = generated_ids[0]
check_token_map = {}

for tk in tokens:
    check_token_map[int(tk)] = tokenizer.decode(tk)

for key, data in check_token_map.items():
    print(f'{key}:', {data})

0: {'<pad>'}
1: {'<s>'}
32099: {'<extra_id_0>'}
39: {'C'}
3060: {'UID'}
23623: {'lg'}
2828: {'Impl'}
970: {'ant'}
5664: {'Lib'}
18: {'.'}
98: {'~'}


In [65]:
print(type(outputs.label_ids[0]), outputs.label_ids[0].shape)
for tokens in outputs.label_ids:
    # print(tokens)
    
    # print(tokenizer.decode(tokens))
    break


tokens = outputs.label_ids[0]
check_token_map = {}
for tk in tokens:
    check_token_map[int(tk)] = tokenizer.decode(tk)

for key, data in check_token_map.items():
    print(f'{key}:', {data})

<class 'numpy.ndarray'> (512,)
1: {'<s>'}
39: {'C'}
3060: {'UID'}
23623: {'lg'}
2828: {'Impl'}
970: {'ant'}
5664: {'Lib'}
385: {' C'}
18: {'.'}
98: {'~'}
1435: {'()'}
7010: {' \n'}
203: {'\n'}
6315: {'._'}
84: {'p'}
12804: {'Img'}
4247: {'Title'}
682: {'List'}
5669: {'Man'}
11853: {'ufact'}
594: {'ure'}
225: {' '}
3163: {'System'}
5895: {'Ab'}
322: {'ut'}
475: {'ment'}
1488: {'Model'}
40: {'D'}
11124: {'Preview'}
2: {'</s>'}
0: {'<pad>'}


In [19]:
train_data_dict_list = []
for idx, info in enumerate(short_method_list):
    data_dict = {}
    data_dict['oms'] = info
    data_dict['code'] = all_code_list[idx]
    data_dict['
    '] = all_tokens_list[idx]
    data_dict['tokens_ids'] = all_tokens_ids_list[idx]
    train_data_dict_list.append(data_dict)
print(len(train_data_dict_list))

46


In [24]:

from collections import defaultdict
BERT_MAX_SIZE = 4098
token_ids = []
token_size_map = defaultdict(list)
max_size = 0
for data in train_data_dict_list:
    token_ids.append(data['tokens_ids'])
    size = len(data['tokens_ids'])
    token_size_map[size].append(data)
    if max_size < size:
        max_size = size

print(max_size)
for size_key, datas in sorted(token_size_map.items()):
    print(f'{size_key} tokens {len(datas)} num')

210
15 tokens 1 num
17 tokens 1 num
23 tokens 1 num
25 tokens 1 num
26 tokens 1 num
28 tokens 1 num
29 tokens 3 num
31 tokens 1 num
34 tokens 2 num
35 tokens 2 num
36 tokens 1 num
37 tokens 1 num
38 tokens 1 num
39 tokens 1 num
49 tokens 1 num
53 tokens 1 num
57 tokens 4 num
59 tokens 2 num
62 tokens 1 num
68 tokens 1 num
72 tokens 1 num
80 tokens 1 num
83 tokens 1 num
89 tokens 1 num
90 tokens 1 num
92 tokens 1 num
95 tokens 1 num
100 tokens 1 num
102 tokens 1 num
109 tokens 1 num
129 tokens 1 num
131 tokens 1 num
139 tokens 2 num
146 tokens 1 num
150 tokens 1 num
186 tokens 1 num
210 tokens 1 num


In [ ]:
tokenizer.